In [ ]:
%%capture
!pip install pyLDAvis
!pip install --upgrade ipykernel
!pip install -U numpy

!pip install multi_rake
!pip install summa
!pip install keybert

## Web Data Scraping

In [ ]:
import requests
from bs4 import BeautifulSoup

import pandas as pd

from datetime import datetime
import pytz

In [ ]:
# set date to pull corresponding day's news
now = datetime.now(pytz.timezone('America/New_York'))
today = now.day
yesterday = today - 1
yesterday2 = today - 2

if yesterday == 0 and now.month in [1, 3, 5, 7, 8, 10, 12]: 
  yesterday = 31
  yesterday2 = 30
elif yesterday2 == 0 and now.month in [1, 3, 5, 7, 8, 10, 12]:
  yesterday2 = 31

elif yesterday == 0 and now.month in [4, 6, 9, 11]: 
  yesterday = 30
  yesterday2 = 29
elif yesterday2 == 0 and now.month in [4, 6, 9, 11]:
  yesterday2 = 30

elif yesterday == 0 and now.month == 2:
  if now.year % 4 == 0:
    yesterday = 29
    yesterday2 = 28
  else:
    yesterday = 28
    yesterday2 = 27
elif yesterday2 == 0 and now.month == 2:
  if now.year % 4 == 0:
    yesterday2 = 29
  else:
    yesterday2 = 28

In [ ]:
# create DataFrame with web scraped data from news sources
def get_news(last3, dates):
  df = pd.concat([get_abc(last3, dates), get_mnt(last3), get_ca(last3), get_forbes(last3), get_healthline(last3), get_evhealth(last3), get_modhealth(last3), get_scidaily(last3), 
                  get_cnn(last3), get_atlantic(last3), get_time(last3), get_reuters()])
  return df

In [ ]:
#@title
def scrape_abc(text):
  meta_l = []
  meta = ''

  spl_text = str(text.find('a', class_='AnchorLink')).split('href="')[1]
  for i in range(len(spl_text)):
    if spl_text[i] == '"':
      break
    meta += spl_text[i]

  meta_l.append(meta)
  meta_l.append(text.find('a', class_='AnchorLink').text)
  meta_l.append(text.find('div', class_='ContentRoll__Desc').text)
  return meta_l

def get_abc(last3=False, dates=[]):
  page = requests.get('https://abcnews.go.com/Health/')
  soup = BeautifulSoup(page.content, "html.parser") 
  r1 = soup.find('section', class_='ContentRoll')
  r2 = r1.find_all('section', 'ContentRoll__Item')

  today = dates[0]
  yesterday = dates[1]
  yesterday2 = dates[2]
  if today < 10:
    today = f'0{today}'
  if yesterday < 10:
    yesterday = f'0{yesterday}'
  if yesterday2 < 10:
    yesterday2 = f'0{yesterday2}'

  l = []
  for i in range(len(r2)):
    d = {}
    timestamp = r2[i].find('div', class_='ContentRoll__TimeStamp').text

    if last3:
      if (str(today) not in timestamp and 'hour' not in timestamp) and str(yesterday) not in timestamp and str(yesterday2) not in timestamp:
        break
    else:
      if str(today) not in timestamp and 'hour' not in timestamp:
        break
      
    meta_l = scrape_abc(r2[i])
    
    d['URL'] = meta_l[0]
    d['Headline'] = meta_l[1]
    d['Description'] = meta_l[2]
    d['Source'] = 'ABC'
    l.append(d)
    d = {}

  return pd.DataFrame(l)

In [ ]:
#@title
def scrape_mnt(text):
  meta_l = []
  meta = 'https://www.medicalnewstoday.com/'

  spl_text = str(text.find('a', class_='css-2fdibo')).split('href="')[1]
  for i in range(len(spl_text)):
    if spl_text[i] == '"':
      break
    meta += spl_text[i]

  meta_l.append(meta)
  meta_l.append(text.find('h2', class_='css-5dw8ta').text)
  meta_l.append(text.find('a', class_='css-2fdibo').text)
  return meta_l

def get_mnt(last3=False):
  page = requests.get('https://www.medicalnewstoday.com/news')
  soup = BeautifulSoup(page.content, "html.parser") 
  r1 = soup.find('ol', class_='css-1iruc8t')
  r2 = r1.find_all('li', 'css-kbq0t')

  l = []
  for i in range(len(r2)):
    d = {}
    timestamp = r2[i].find('div', class_='css-3be604').text
    if last3:
      if ((str(today) + ', ') not in timestamp and 'hour' not in timestamp) and (str(yesterday) + ', ') not in timestamp and (str(yesterday2) + ', ') not in timestamp:
        break
    else:
      if (str(today) + ',') not in timestamp:
        break

    meta_l = scrape_mnt(r2[i])
    
    d['URL'] = meta_l[0]
    d['Headline'] = meta_l[1]
    d['Description'] = meta_l[2]
    d['Source'] = 'MedicalNewsToday'
    l.append(d)
    d = {}

  return pd.DataFrame(l)

In [ ]:
#@title
def scrape_ca(text):
  meta_l = []
  meta = ''

  spl_text = str(text.find('h3', class_='title')).split('href="')[1]
  for i in range(len(spl_text)):
    if spl_text[i] == '"':
      break
    meta += spl_text[i]

  meta_l.append(meta)
  meta_l.append(text.find('h3', class_='title').text.strip())
  meta_l.append(text.find('div', class_='content').text.strip())
  return meta_l

def get_ca(last3=False):
  HEADERS = {'User-Agent': 'Mozilla/5.0 (iPad; CPU OS 12_2 like Mac OS X) AppleWebKit/605.1.15 (KHTML, like Gecko) Mobile/15E148'}
  page = requests.get('https://www.clinicaladvisor.com/home/news/', headers=HEADERS)
  soup = BeautifulSoup(page.content, "html.parser")
  r1 = soup.find('div', class_='hm-gutenberg-block hm-content-block -dynamic')
  r2 = r1.find_all('article')   

  l = []
  for i in range(len(r2)):
    d = {}
    timestamp = r2[i].find('time', class_='post-time -published').text
    if last3:
      if ((str(today) + ', ') not in timestamp and 'hour' not in timestamp) and (str(yesterday) + ', ') not in timestamp and (str(yesterday2) + ', ') not in timestamp:
        break
    else:
      if (str(today) + ',') not in timestamp:
        break

    meta_l = scrape_ca(r2[i])
    
    d['URL'] = meta_l[0]
    d['Headline'] = meta_l[1]
    d['Description'] = meta_l[2]
    d['Source'] = 'ClinicalAdvisor'
    l.append(d)
    d = {}

  return pd.DataFrame(l)

In [ ]:
#@title
def scrape_forbes(text):
  meta_l = []
  meta = ''

  spl_text = str(text.find('a', class_='_5ncu0TWl')).split('href="')[1]
  for i in range(len(spl_text)):
    if spl_text[i] == '"':
      break
    meta += spl_text[i]

  meta_l.append(meta)
  meta_l.append(text.find('a', '_5ncu0TWl').text)
  meta_l.append(text.find('p', 'A7hAxSNa').text)
  return meta_l

def get_forbes(last3=False):
  page = requests.get('https://www.forbes.com/healthcare/?sh=5af135ef3b4e')
  soup = BeautifulSoup(page.content, "html.parser")
  r1 = soup.find('div', class_='COe59')
  r2 = r1.find_all('div', class_='B6j66vzQ')
  
  l = []
  for i in range(len(r2)):
    d = {}
    timestamp = r2[i].find('span', class_='_9u4PrQql JJ0p5dnD').text
    if last3:
      if ('hour' not in timestamp and 'minute' not in timestamp) and (str(yesterday) + ', ') not in timestamp and (str(yesterday2) + ', ') not in timestamp:
        break
    else:
      if 'hour' not in timestamp and 'minute' not in timestamp:
        break
    
    meta_l = scrape_forbes(r2[i])
    
    d['URL'] = meta_l[0]
    d['Headline'] = meta_l[1]
    d['Description'] = meta_l[2]
    d['Source'] = 'Forbes'
    l.append(d)
    d = {}

  return pd.DataFrame(l)

In [ ]:
#@title
def scrape_healthline(text):
  meta_l = []
  meta = 'https://www.healthline.com'

  spl_text = str(text.find('a', class_='css-2fdibo')).split('href="')[1]
  for i in range(len(spl_text)):
    if spl_text[i] == '"':
      break
    meta += spl_text[i]

  meta_l.append(meta)
  meta_l.append(text.find('h2', 'css-1jcjjjn').text)
  meta_l.append(text.find('a', 'css-2fdibo').text)
  return meta_l

def get_healthline(last3=False):
  page = requests.get('https://www.healthline.com/health-news')
  soup = BeautifulSoup(page.content, "html.parser")
  r1 = soup.find('ol', class_='css-1iruc8t')
  r2 = r1.find_all('li', class_='css-18vzruc')

  l = []
  for i in range(len(r2)):
    d = {}
    timestamp = r2[i].find('div', class_='css-mmjpxh').text
    if last3:
      if (str(today) + ',') not in timestamp and (str(yesterday) + ', ') not in timestamp and (str(yesterday2) + ', ') not in timestamp:
        break
    else:
      if (str(today) + ',') not in timestamp:
        break

    meta_l = scrape_healthline(r2[i])
    
    d['URL'] = meta_l[0]
    d['Headline'] = meta_l[1]
    if not meta_l[2][-1].isalnum():
      d['Description'] = meta_l[2][:-1]
    else:
      d['Description'] = meta_l[2]
    d['Source'] = 'Healthline'
    l.append(d)
    d = {}

  return pd.DataFrame(l)

In [ ]:
#@title
def scrape_evhealth(text):
  meta_l = []
  meta = ''

  spl_text = str(text.find('a', class_='cr-anchor')).split('href="')[1]
  for i in range(len(spl_text)):
    if spl_text[i] == '"':
      break
    meta += spl_text[i]

  meta_l.append(meta)
  meta_l.append(text.find('a', 'cr-anchor').text)
  meta_l.append(text.find('div', 'category-index-article__dek').text)
  return meta_l

def get_evhealth(last3=False):
  page = requests.get('https://www.everydayhealth.com/news/')
  soup = BeautifulSoup(page.content, "html.parser")
  r1 = soup.find('section', class_='category-articles')
  r2 = r1.find('article', class_='category-index-article category-index-article--large')
  r3 = r1.find_all('article', class_='category-index-article category-index-article--regular')

  timestamp = r2.find('span', class_='category-index-article__date').text
  l = []
  if (str(today) + ',') in timestamp or (str(yesterday) + ', ') not in timestamp or (str(yesterday2) + ', ') not in timestamp:
    d = {}
    meta_l = scrape_evhealth(r2)
    d['URL'] = meta_l[0]
    d['Headline'] = meta_l[1]
    d['Description'] = meta_l[2]
    d['Source'] = 'EverydayHealth'
    l.append(d)

    for i in range(len(r3)):
      d = {}
      timestamp = r3[i].find('span', class_='category-index-article__date').text
      if last3:
        if (str(today) + ',') not in timestamp and (str(yesterday) + ', ') not in timestamp and (str(yesterday2) + ', ') not in timestamp:
          break
      else:
        if (str(today) + ',') not in timestamp:
          break

      meta_l = scrape_evhealth(r3[i])
      
      d['URL'] = meta_l[0]
      d['Headline'] = meta_l[1]
      d['Description'] = meta_l[2]
      d['Source'] = 'EverydayHealth'
      l.append(d)
      d = {}

  return pd.DataFrame(l)

In [ ]:
#@title
def scrape_modhealth(text):
  meta_l = []
  meta = ''

  spl_text = str(text.find('h3', class_='middle-article-headline')).split('href="')[1]
  for i in range(len(spl_text)):
    if spl_text[i] == '"':
      break
    meta += spl_text[i]

  meta_l.append(meta)
  meta_l.append(text.find('h3', class_='middle-article-headline').text.strip())
  meta_l.append(text.find('div', class_='feature-article-summary pparagraph').text.strip())
  return meta_l

def get_modhealth(last3=False):
  page = requests.get('https://www.modernhealthcare.com/')
  soup = BeautifulSoup(page.content, "html.parser")
  r1 = soup.find('div', class_='block-region-content-left')
  r2 = r1.find('div', class_='feature-article-headline-wrapper col-md-5 col-lg-5 col-xs-12 col-sm-12')
  r3 = r1.find_all('div', class_='middle-article-block')

  url = 'https://www.modernhealthcare.com'
  spl_text = str(r2.find('div', class_='feature-article-headline')).split('href="')[1]
  for i in range(len(spl_text)):
    if spl_text[i] == '"':
      break
    url += spl_text[i]

  l = []
  d = {}
  d['URL'] = url
  d['Headline'] = r2.find('div', class_='feature-article-headline').text.strip()
  d['Description'] = r2.find('div', class_='feature-article-summary').text.strip()
  d['Source'] = 'ModernHealthcare'
  l.append(d)

  if last3:
    limit = 10
  else: 
    limit = 4
  for i in range(limit):
    d = {}
    meta_l = scrape_modhealth(r3[i])
    
    d['URL'] = meta_l[0]
    d['Headline'] = meta_l[1]
    d['Description'] = meta_l[2]
    d['Source'] = 'ModernHealthcare'
    l.append(d)
    d = {}

  return pd.DataFrame(l)

In [ ]:
#@title
def scrape_scidaily(text):
  meta_l = []
  meta = 'https://www.sciencedaily.com/'

  spl_text = str(text.find('h3', class_='latest-head')).split('href="')[1]
  for i in range(len(spl_text)):
    if spl_text[i] == '"':
      break
    meta += spl_text[i]

  meta_l.append(meta)
  meta_l.append(text.find('h3', class_='latest-head').text.strip())
  meta_l.append(text.find('div', class_='latest-summary').text.split('—')[1].strip())
  return meta_l

def get_scidaily(last3):
  HEADERS = {'User-Agent': 'Mozilla/5.0 (iPad; CPU OS 12_2 like Mac OS X) AppleWebKit/605.1.15 (KHTML, like Gecko) Mobile/15E148'}
  page = requests.get('https://www.sciencedaily.com/news/top/health/', headers=HEADERS)
  soup = BeautifulSoup(page.content, "html.parser")
  r1 = soup.find('div', class_='hero tab-content')
  r2 = r1.find_all('div', class_='tab-pane')

  l = []
  for i in range(len(r2)):
    d = {}
    timestamp = r2[i].find('span', class_='story-date').text
    if last3:
      if (str(today) + ',') not in timestamp and (str(yesterday) + ', ') not in timestamp and (str(yesterday2) + ', ') not in timestamp:
        break
    else:
      if (str(today) + ',') not in timestamp:
        break

    meta_l = scrape_scidaily(r2[i])
    
    d['URL'] = meta_l[0]
    d['Headline'] = meta_l[1]
    if '...' in meta_l[2]:
      d['Description'] = meta_l[2][:-4]
    else:
      d['Description'] = meta_l[2]
    d['Source'] = 'Healthline'
    l.append(d)
    d = {}

  return pd.DataFrame(l)

In [ ]:
#@title
def scrape_cnn(text):
  meta_l = []
  meta = 'https://www.cnn.com/health'

  spl_text = str(text.find('a', class_='container__link container_lead-plus-headlines__link')).split('href="')[1]
  for i in range(len(spl_text)):
    if spl_text[i] == '"':
      break
    meta += spl_text[i]

  meta_l.append(meta)
  meta_l.append(text.find('div', class_='container__headline container_lead-plus-headlines__headline').text.strip())
  meta_l.append('')
  return meta_l

def get_cnn(last3=False):
  page = requests.get('https://www.cnn.com/health')
  soup = BeautifulSoup(page.content, "html.parser")
  r1 = soup.find('div', class_='container__field-links container_lead-plus-headlines__field-links')
  r2 = r1.find_all('div', class_='card container__item container__item--type-section container_lead-plus-headlines__item container_lead-plus-headlines__item--type-section')  

  l = []
  for i in range(1, len(r2)):
    d = {}
    meta_l = scrape_cnn(r2[i])
    timestamp = meta_l[0].split('/')[6]

    if last3:
      if str(today) not in timestamp and str(yesterday) not in timestamp and str(yesterday2) not in timestamp:
        break
    else:
      if str(today) not in timestamp:
        break
    
    d['URL'] = meta_l[0]
    d['Headline'] = meta_l[1]
    d['Description'] = meta_l[2]
    d['Source'] = 'CNN'
    l.append(d)
    d = {}

  return pd.DataFrame(l)

In [ ]:
#@title
def scrape_atlantic(text):
  meta_l = []
  meta = ''

  spl_text = str(text).split('href="')[1]
  for i in range(len(spl_text)):
    if spl_text[i] == '"':
      break
    meta += spl_text[i]

  meta_l.append(meta)
  meta_l.append(text.find('h2', class_='LandingRiver_title__4ibQ4').text)
  meta_l.append(text.find('p', class_='LandingRiver_dek__u9vaI').text)
  return meta_l

def get_atlantic(last3=False):
  page = requests.get('https://www.theatlantic.com/health/')
  soup = BeautifulSoup(page.content, "html.parser")
  r1 = soup.find('ul', class_='LandingRiver_ul__nRR9m')
  r2 = r1.find_all('li', class_='LandingRiver_li__Db7WD')

  l = []
  if last3:
    limit = 6
  else: 
    limit = 2
  for i in range(limit):
    d = {}    
    meta_l = scrape_atlantic(r2[i])

    d['URL'] = meta_l[0]
    d['Headline'] = meta_l[1]
    d['Description'] = meta_l[2]
    d['Source'] = 'Atlantic'
    l.append(d)
    d = {}

  return pd.DataFrame(l)

In [ ]:
#@title
def scrape_time(text):
  meta_l = []
  meta = 'https://time.com'

  spl_text = str(text.find('a')).split('href="')[1]
  for i in range(len(spl_text)):
    if spl_text[i] == '"':
      break
    meta += spl_text[i]

  meta_l.append(meta)
  meta_l.append(text.find('h2', class_='headline').text.strip())
  if text.find('h3', class_='summary') != None:
    meta_l.append(text.find('h3', class_='summary').text.strip())
  else:
    meta_l.append('')
  return meta_l

def get_time(last3=False):
  page = requests.get('https://time.com/section/health/')
  soup = BeautifulSoup(page.content, "html.parser")
  r1 = soup.find_all('div', class_='taxonomy-tout')

  l = []
  for i in range(len(r1)):
    d = {}
    if last3 and i >= 4:
      timestamp = r1[i].find('span', class_='byline').text
      if (str(today) + ',') not in timestamp and (str(yesterday) + ', ') not in timestamp and (str(yesterday2) + ', ') not in timestamp:
        break
    else:
      if i == 4:
        break
    
    meta_l = scrape_time(r1[i])
    
    d['URL'] = meta_l[0]
    d['Headline'] = meta_l[1]
    d['Description'] = meta_l[2]
    d['Source'] = 'Time'
    l.append(d)
    d = {}

  return pd.DataFrame(l)

In [ ]:
#@title
def scrape_reuters(text):
  meta_l = []
  meta = 'https://www.reuters.com'

  spl_text = str(text).split('href="')[1]
  for i in range(len(spl_text)):
    if spl_text[i] == '"':
      break
    meta += spl_text[i]

  meta_l.append(meta)
  meta_l.append(text.find('a', class_='text__text__1FZLe text__dark-grey__3Ml43 text__medium__1kbOh text__heading_5_and_half__3YluN heading__base__2T28j heading_5_half title__title__1Sa2a title').text)
  meta_l.append('')
  return meta_l

def get_reuters():
  page = requests.get('https://www.reuters.com/business/healthcare-pharmaceuticals/')
  soup = BeautifulSoup(page.content, "html.parser")
  r1 = soup.find('div', class_='static-media-maximizer__hero__3I-8O')
  r2 = soup.find_all('li', class_='static-media-maximizer__card__3Ke0W static-media-maximizer__list__23N8R')

  url = 'https://www.reuters.com'
  spl_text = str(r1).split('href="')[1]
  for i in range(len(spl_text)):
    if spl_text[i] == '"':
      break
    url += spl_text[i]

  l = []
  d = {}
  d['URL'] = url
  d['Headline'] = r1.find('a', class_='text__text__1FZLe text__dark-grey__3Ml43 text__medium__1kbOh text__heading_3__1kDhc heading__base__2T28j heading__heading_3__3aL54 title__title__1Sa2a hero-card__title__33EFM').text
  d['Description'] = r1.find('p', class_='text__text__1FZLe text__dark-grey__3Ml43 text__regular__2N1Xr text__body__yKS5U body__base__22dCE body__body__VgU9Q hero-card__description__2Ih7f').text
  d['Source'] = 'Reuters'
  l.append(d)

  for i in range(len(r2)):
    d = {}
    meta_l = scrape_reuters(r2[i])
    
    d['URL'] = meta_l[0]
    d['Headline'] = meta_l[1]
    d['Description'] = meta_l[2]
    d['Source'] = 'Reuters'
    l.append(d)
    d = {}

  return pd.DataFrame(l)

In [ ]:
# set flag to get news from last 3 days
last3 = True
dates = [today, yesterday, yesterday2]

df = get_news(last3, dates).reset_index(drop=True)
df['Full_Text'] = df['Headline'] + '. ' + df['Description']
df

## Topic Modeling

In [ ]:
import gensim
from gensim.utils import simple_preprocess
from gensim.corpora import Dictionary
from gensim.models.ldamodel import LdaModel
from gensim.models.coherencemodel import CoherenceModel
from gensim.models import Phrases
from gensim.models.phrases import Phraser

import re
import spacy
import nltk
from nltk.tokenize import RegexpTokenizer
nltk.download('stopwords')
from nltk.corpus import stopwords

import pyLDAvis
from pyLDAvis.gensim_models import prepare
pyLDAvis.enable_notebook()

In [ ]:
texts = df['Full_Text'].to_list()

# pre-process text to remove extraneous symbols
texts = [re.sub('\s+', ' ', text) for text in texts]
texts = [re.sub("\'", "", text) for text in texts]
texts = [re.sub("-", " ", text) for text in texts]
texts = [re.sub(":", "", text) for text in texts]

In [ ]:
stops = set(stopwords.words('english'))
words = []

# parse words from text and remove stopwords
for text in texts:
  words.append([word for word in simple_preprocess(str(text), deacc=True) if word not in stops])

#nlp = spacy.load('en_core_web_sm', disable=['parser', 'ner'])
nlp = spacy.load('en_core_web_sm')

In [ ]:
# create sequences of 2 and 3 words
def make_ngrams(texts):
  bigram = Phrases(words, min_count=5, threshold=100)
  bigram_mod = Phraser(bigram)
  bigrams = [bigram_mod[text] for text in texts]

  trigram = Phrases(bigram[words], threshold=100)
  trigram_mod = Phraser(trigram)  
  trigrams = [trigram_mod[bigram_mod[text]] for text in texts]
  return bigrams, trigrams

# break down words to root forms
def lemmatize(texts, allowed_postags=['NOUN', 'ADJ', 'VERB', 'ADV']):
  ntexts = []
  for text in texts:
    doc = nlp(" ".join(text)) 
    ntexts.append([token.lemma_ for token in doc if token.pos_ in allowed_postags])
  return ntexts

In [ ]:
bigram_texts = make_ngrams(words)[0]
corpus = lemmatize(bigram_texts)

# get term frequency vector
id2word = Dictionary(corpus)
tdf = [id2word.doc2bow(text) for text in corpus]

In [ ]:
# fit LDA model on corpus with variable number of topics
coh_values = []
models = []
for num_topics in range(2, 21):
  lda = LdaModel(corpus=tdf, id2word=id2word, num_topics=num_topics, random_state=100, update_every=1, chunksize=100, passes=10, alpha='auto', per_word_topics=True)
  models.append(lda)
  coh_model = CoherenceModel(model=lda, texts=corpus, dictionary=id2word, coherence='c_v')
  coh_values.append(coh_model.get_coherence())

In [ ]:
# select best LDA model based on coherence score
idx = coh_values.index(max(coh_values))
num_topics = idx + 2
best_lda = models[idx]

In [ ]:
print(f"Perplexity: {best_lda.log_perplexity(tdf)}")
print(f"Coherence Score: {coh_values[idx]}")
best_lda.print_topics()

In [ ]:
# visualize topic distribution
vis = prepare(best_lda, tdf, id2word)
vis

In [ ]:
# create DataFrame to find each article's dominant topic
l = []
for i, r in enumerate(best_lda[tdf]):
  d = {}
  dom_topic = r[0][0][0]
  kwords = []
  for x in best_lda.show_topic(dom_topic):
    kwords.append(x[0])

  d['Dominant_Topic'] = dom_topic
  d['Dominant_Topic_Keywords'] = ', '.join(kwords)
  d['Headline'] = df.iloc[i]['Headline']
  d['URL'] = df.iloc[i]['URL']
  l.append(d)

ddf = pd.DataFrame(l)
ddf

## Keyword Extraction

In [ ]:
from multi_rake import Rake
from keybert import KeyBERT

from collections import Counter
import seaborn as sns
import matplotlib.pyplot as plt

In [ ]:
%%capture
texts = df['Full_Text'].to_list()

rake = Rake()
keybert = KeyBERT(model='all-mpnet-base-v2')

In [ ]:
# extract keywords from corpus using separate approaches
rake_kwords = []
bert_kwords = []
for text in texts:
  rake_text = rake.apply(text)
  if rake_text != []:
    rake_kwords.append(rake_text)
  else:
    rake_kwords.append('')
  bert_kwords.append(list(dict(keybert.extract_keywords(text, keyphrase_ngram_range=(1, 2), stop_words='english', use_mmr=True, top_n=10)).keys()))

In [ ]:
# create DataFrame to get each article's individual keywords, verified by both approaches
l = []
for i in range(len(bert_kwords)):
  d = {'Article_Keywords': []}
  d['Headline'] = df.iloc[i]['Headline']
  d['URL'] = df.iloc[i]['URL']

  for kword in bert_kwords[i]:
    token = nlp(kword)
    for kword2 in rake_kwords[i]:
      # add word as keyword if found by both approaches
      if kword == kword2[0]:
        d['Article_Keywords'].append(kword)
      # add word as keyword if semantically similar to any word found by other approach
      else:
        token2 = nlp(kword2[0])
        score = token.similarity(token2)
        if score >= 0.8:
          d['Article_Keywords'].append(kword)

  l.append(d)

ddf2 = pd.DataFrame(l)
ddf2

In [ ]:
all_kwords = []
for i, r in ddf2.iterrows():
  all_kwords.extend(r['Article_Keywords'])

counter = Counter(all_kwords)

In [ ]:
# plot top k keywords for days' news articles
k = 20
topk_kwords = counter.most_common(k)

x = []
y = []
for kword in topk_kwords:
  x.append(kword[0])
  y.append(kword[1])

if last3:
  plt.figure(figsize=(k*1.5, k*0.75))
  plt.title(f"Top {k} Keywords of the Last 3 Days", fontdict={'fontsize': k*1.5})
  plt.tight_layout()
  _ = sns.barplot(x=x, y=y)
else:
  plt.figure(figsize=(k*1.5, k*0.75))
  plt.title(f"Top {k} Keywords of the Day", fontdict={'fontsize': k*1.5})
  plt.tight_layout()
  _ = sns.barplot(x=x, y=y)